In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0MB 3.9MB/s 
     |████████████████████████████████| 6.5MB 19.4MB/s 
     |████████████████████████████████| 15.3MB 256kB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
  Found existing installation: click 8.0.0
    Uninstalling click-8.0.0:
      Successfully uninstalled click-8.0.0


In [ ]:
import warnings
warnings.filterwarnings('ignore')
 
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

### Задача:

Прогноз продаж одной из популярных моделей [фичерфонов](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D1%87%D0%B5%D1%80%D1%84%D0%BE%D0%BD) (на картинке ниже пример похожего устройства) в салонах МегаФона
![](https://39.img.avito.st/640x480/8468720439.jpg)

### Исходные данные:

Датасет содержит следующие поля:

1. `point_id` - Индентификатор салона
2. `lon` - Долгота точки
3. `lat` - Широта точки
4. `target` - Значение таргета, усредненное за несколько месяцев и отнормированное

### Требования к решению и советы:

Ниже приведен список из нескольких важных пунктов, необходимых для решения задания. Выполнение каждого из пунктов влияет на итоговую оценку. Вы можете выполнить каждый из пунктов разными способами, самым лучшим будет считаться вариант, когда всё получение и обработка данных будут реализованы на Питоне (пример: вы можете скачать данные из OSM через интерфейс на сайте overpass-turbo или с помощью библиотек `overpass`/`requests`. Оба варианта будут зачтены, но больше баллов можно заработать во втором случае)



1. Салоны расположены в нескольких разных городах, вам необходимо **определить город для каждого салона** (это понадобится во многих частях задания). К этому есть разные подходы. Вы можете провести [обратное геокодирование](https://en.wikipedia.org/wiki/Reverse_geocoding) с помощью геокодера [Nominatim](https://nominatim.org/), доступного через библиотеку `geopy` примерно вот так:
```python
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)
```
В таком случае, вам придется обрабатывать полученную строку адреса, чтобы извлечь название города. Также вы можете скачать из OSM или найти в любом другом источнике границы административно территориальных границ России и пересечь с ними датасет с помощью `geopandas.sjoin` (этот вариант более надежный, но нужно будет разобраться с тем, как устроены границы АТД в OSM, обратите внимание на [этот тег](https://wiki.openstreetmap.org/wiki/Key:admin_level))


2. **Используйте данные OSM**: подумайте, какие объекты могут влиять на продажи фичерфонов. Гипотеза: такие телефоны покупают люди, приезжающие в город или страну ненадолго, чтобы вставить туда отдельную симкарту для роуминга. Можно попробовать использовать местоположения железнодорожных вокзалов (изучите [этот тег](https://wiki.openstreetmap.org/wiki/Tag:railway%3Dstation)). Необходимо использовать хотя бы 5 разных типов объектов из OSM. Скорее всего, вам придется качать данные OSM отдельно для разных городов (см. пример для Нью-Йорка из лекции)


3. **Используйте разные способы генерации признаков**: описать положение салона МегаФона относительно станций метро можно разными способами - найти ***расстояние до ближайшей станции***, или же посчитать, сколько станций попадает в ***500 метровую буферную зону*** вокруг салона. Такие признаки будут нести разную информацию. Так же попробуйте поэкспериментировать с размерами буферных зон (представьте, что значат в реальности радиусы 100, 500, 1000 метров). Попробуйте посчитать расстояние до центра города, до других объектов.

4. **Сделайте визуализации**: постройте 2-3 карты для какого нибудь из городов - как распределен в пространстве таргет, где находятся объекты, полученные вами из OSM. Можете использовать любой инструмент - обычный `plot()`, `folium`, `keplergl`. Если выберете Кеплер, обязательно сохраните в файл конфиг карты, чтобы ее можно было воспроизвести. Сделать это можно вот так:

```python
import json
json_data = kepler_map.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)
```
5. Задание не ограничено приведенными выше пунктами, попробуйте нагенерировать интересных признаков, найти в интернете дополнительные данные (в таком случае в комментарии к коду укажите ссылку на ресурс, откуда взяли данные)



6. Это довольно сложная задача - датасет очень маленький, данные по своей природе довольно случайны. Поэтому место и скор на Kaggle не будут играть решающую роль в оценке, но позволят заработать дополнительные баллы

In [ ]:
!mkdir ./data
!wget https://raw.githubusercontent.com/dmasny99/techpark2021/main/hometasks/hometask%206/mf_geo_train.csv -O ./data/mf_geo_train.csv
!wget https://raw.githubusercontent.com/dmasny99/techpark2021/main/hometasks/hometask%206/mf_geo_test.csv -O ./data/mf_geo_test.csv

--2021-05-23 09:03:11--  https://raw.githubusercontent.com/dmasny99/techpark2021/main/hometasks/hometask%206/mf_geo_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23272 (23K) [text/plain]
Saving to: ‘./data/mf_geo_train.csv’

./data/mf_geo_train 100%[===================>]  22.73K  --.-KB/s    in 0.003s  

2021-05-23 09:03:11 (8.44 MB/s) - ‘./data/mf_geo_train.csv’ saved [23272/23272]

--2021-05-23 09:03:12--  https://raw.githubusercontent.com/dmasny99/techpark2021/main/hometasks/hometask%206/mf_geo_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, 

### Read data

In [ ]:
train = pd.read_csv('./data/mf_geo_train.csv')
test = pd.read_csv('./data/mf_geo_test.csv')

In [ ]:
train.head(2)

,point_id,lon,lat,target
0,ommNZCUV,37.590776,55.848630,-0.348157
1,nMe2LHPb,37.784210,55.750271,1.294206


In [ ]:
test.head(2)

,point_id,lon,lat,target
0,F4lXR1cG,37.681242,55.74804,0.0091
1,4LJu4GTf,60.580910,56.79586,0.0091


## 1) Определение города для каждого салона

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="https")
for i in range(train.shape[0]//20):
  location = geolocator.reverse(str(train.lat[i])+','+ str(train.lon[i]))
  print(location.address.split(','))

['6А', ' Сигнальный проезд', ' район Отрадное', ' Москва', ' Центральный федеральный округ', ' 127106', ' Россия']
['38/18', ' 2-я Владимирская улица', ' Новогиреево', ' район Новогиреево', ' Москва', ' Центральный федеральный округ', ' 4А', ' Россия']
['18', ' улица Зорге', ' ЗЖМ', ' Советский район', ' Ростов-на-Дону', ' городской округ Ростов-на-Дону', ' Ростовская область', ' Южный федеральный округ', ' 344000', ' Россия']
['Торговый центр "Серебряный Дом"', ' 16', ' Большая Семёновская улица', ' Семёновское', ' район Соколиная Гора', ' Москва', ' Центральный федеральный округ', ' 105094', ' Россия']
['5', ' Кожевническая улица', ' район Замоскворечье', ' Москва', ' Центральный федеральный округ', ' 115280', ' Россия']
['12', ' улица 9 Мая', ' Северный', ' 3-ий микрорайон', ' Советский район', ' Красноярск', ' городской округ Красноярск', ' Красноярский край', ' Сибирский федеральный округ', ' 660000', ' Россия']
['Ледовый дворец', ' 1', ' проспект Пятилеток', ' округ Правобережный

У других городов (не федерального значения) на 4ой позиции с конца стоит название области или края. Поэтому перепишу код с учетом этого.

In [ ]:
fed_cities =[' Москва',' Санкт-Петербург']
cities = []
for i in range(train.shape[0]):
  location = geolocator.reverse(str(train.lat[i])+','+ str(train.lon[i]))
  loc_list = location.address.split(',')
  if loc_list[-4] in fed_cities:
    cities.append(loc_list[-4])
  else:
    cities.append(loc_list[-5].split()[-1])

In [ ]:
cities = list(map(lambda x: x.strip(),cities))

In [ ]:
train['city'] = cities

In [ ]:
#удалю балашиху
train.city.value_counts()

Москва             160
Санкт-Петербург     83
Самара              27
Новосибирск         26
Казань              25
Екатеринбург        22
Ростов-на-Дону      21
Новгород            21
Красноярск          20
Уфа                 19
Балашиха             1
Name: city, dtype: int64

In [ ]:
train.drop(index = train[train.city == 'Балашиха'].index,inplace= True)

Сразу начну собирать эту предобработку в пайплайн, чтобы потом все то же самое проедлать для теста. Код вверху не убираю,чтобы было обоснование действий.

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="https")
from sklearn.base import BaseEstimator, TransformerMixin
class CityName(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        fed_cities =[' Москва',' Санкт-Петербург']
        cities = []
        for i in range(X.shape[0]):
          location = geolocator.reverse(str(X.lat[i])+','+ str(X.lon[i]))
          loc_list = location.address.split(',')
          if loc_list[-4] in fed_cities:
            cities.append(loc_list[-4])
          else:
            cities.append(loc_list[-5].split()[-1]) 
        cities = list(map(lambda x: x.strip(),cities))
        X['city'] = cities
        X.drop(index = X[X.city == 'Балашиха'].index,inplace= True)
        X.reset_index(inplace = True)
        return X


In [ ]:

from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('get_city_name', CityName())])


Перед этим шагом заново запустить ячейку, где трейн и тест создаются из csv файла, чтобы нижняя ячейка не упала! Можно было бы отдельно все для теста проделать и не собирать в пайплайн, но пока так.

In [ ]:
# перезаписал тест и прогнал тест и трейн через пайплайн
test = pipeline.fit_transform(test)
train = pipeline.fit_transform(train)

## 2) Данные из OSM

Далее нахожу ID каждого уникального города из датасета,чтобы в запросах к OSM мне возвращались только те amenities, которые находятся в интересующем меня городе.

In [ ]:
!pip install OSMPythonTools

     |████████████████████████████████| 184kB 3.9MB/s 
  Created wheel for OSMPythonTools: filename=OSMPythonTools-0.3.0-cp37-none-any.whl size=13025 sha256=7a78930b90947dcb6b85a2c9428dc5aa7a2e4382273d8559cfc5474226f4eb96
  Stored in directory: /root/.cache/pip/wheels/8b/5b/c9/808660b16cb9745083a31339c69ec9776e09eb40f2164c6565
  Created wheel for pytest-sugar: filename=pytest_sugar-0.9.4-py2.py3-none-any.whl size=8971 sha256=ce2f6c18536050398b4a4bf706a6d4d982a9d1a93f041470b1a2075d8958e7da
  Stored in directory: /root/.cache/pip/wheels/72/ac/7f/55050ee786fa4a2bb2d05dea0696eab826ff1d5b8a7dbd5883
Successfully built OSMPythonTools pytest-sugar


In [ ]:
from OSMPythonTools.nominatim import Nominatim
nom = Nominatim()

In [ ]:
unique_cities = train.city.unique()

In [ ]:
unique_cities

array(['Москва', 'Ростов-на-Дону', 'Красноярск', 'Санкт-Петербург', 'Уфа',
       'Казань', 'Екатеринбург', 'Новгород', 'Новосибирск', 'Самара'],
      dtype=object)

In [ ]:
cities_id = list(map(lambda x: nom.query(x).areaId(),unique_cities))

[nominatim] downloading data: search
[nominatim] downloading data: search
[nominatim] downloading data: search
[nominatim] downloading data: search
[nominatim] downloading data: search
[nominatim] downloading data: search
[nominatim] downloading data: search
[nominatim] downloading data: search
[nominatim] downloading data: search


In [ ]:
cities_dict = dict(zip(unique_cities,cities_id))

In [ ]:
cities_dict

{'Екатеринбург': 3606564910,
 'Казань': 3603437391,
 'Красноярск': 3601430616,
 'Москва': 3602555133,
 'Новгород': 3602417529,
 'Новосибирск': 3600366544,
 'Ростов-на-Дону': 3601285772,
 'Самара': 3603368701,
 'Санкт-Петербург': 3600421007,
 'Уфа': 3601549169}

In [ ]:
for i in cities_dict.keys():
  print(cities_dict[i])

3602555133
3601285772
3601430616
3600421007
3601549169
3603437391
3606564910
3602417529
3600366544
3603368701


Тк для трейна и теста уникальные города совпадают, то проделываю эту операцию лишь один раз.

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

Буду смотреть следующие места: метро и жд вокзалы, фастфуд, хостелы, достопримечательности, ТЦ. П.С. хотел взять бизнес центры, но по тэгу building = commercial, как указано на вики осм, пустой ответ, наверное, информация о БЦ не занесена нужным образом в ОСМ про города, которые тут используются.

In [ ]:
def osm_request(id):
  tmp_dict = {}
  places = {'station':'railway','fast_food':'amenity','hostel':'tourism','attraction':'tourism','mall':'shop'} # ключ значение поменяны, тк 2 туризма
  for i in places.keys():
    overpass_query = """
    [out:json];
    area({});
    node[{}={}](area);
    out center;
    """.format(id,places[i],i)
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    tmp_dict[i] = response.json()
  return tmp_dict
  

Для каждого города делаю запрос к OSM и записываю полученные данные в словарь.

In [ ]:
ngor_osm = osm_request(cities_dict['Новгород'])

In [ ]:
ekb_osm = osm_request(cities_dict['Екатеринбург'])

In [ ]:
kzn_osm = osm_request(cities_dict['Казань'])

In [ ]:
krsn_osm = osm_request(cities_dict['Красноярск'])

In [ ]:
msc_osm = osm_request(cities_dict['Москва'])

In [ ]:
nsib_osm = osm_request(cities_dict['Новосибирск'])

In [ ]:
rnd_osm = osm_request(cities_dict['Ростов-на-Дону'])

In [ ]:
sam_osm = osm_request(cities_dict['Самара'])

In [ ]:
spb_osm =osm_request(cities_dict['Санкт-Петербург'])

In [ ]:
ufa_osm = osm_request(cities_dict['Уфа'])

P.S. Иногда, код в ячейках выше валится, похоже,что я иногда получаю пустые ответы на запросы (не знаю, почему). Лечится повторным вызовом функции (ну или до тех пор, пока не отработает нормально она). А вообще интересно, почему так происходит. По идее можно поставить какой ниудь отлавливатель в саму функцию, который бы сам проверял, получил он ответ с кодом 200 или нет и повтоярб лы, если там не 200. Но я на это забил и пошел дальше.

## 3) Генерация признаков

### 3.1) Первым признаком будет расстояни от центра города до точки продаж. Центр города будем искать через метод объекта геолокатор.

Получаем кортеж в формате (lat,lon)

In [ ]:
centers = []
for city in unique_cities:
  centers.append(geolocator.geocode(city)[-1])

In [ ]:
cities_centers =dict(zip(unique_cities,centers))

In [ ]:
cities_centers

{'Екатеринбург': (56.839104, 60.60825),
 'Казань': (55.7823547, 49.1242266),
 'Красноярск': (56.0090968, 92.8725147),
 'Москва': (55.7504461, 37.6174943),
 'Новгород': (58.5209862, 31.2757862),
 'Новосибирск': (54.96781445, 82.95159894278376),
 'Ростов-на-Дону': (47.2213858, 39.7114196),
 'Самара': (53.198627, 50.113987),
 'Санкт-Петербург': (59.938732, 30.316229),
 'Уфа': (46.370935349999996, 6.231168493722434)}

Здесь неверно определены координаты уфы, он ее транслитирирует на UEFA и выдет координаты в Женеве. Поэтому вручную найду координаты для Уфы и перезапишу их в словарь.

In [ ]:
cities_centers['Уфа'] =geolocator.geocode('Уфа Башкортостан')[-1]

In [ ]:
cities_centers

{'Екатеринбург': (56.839104, 60.60825),
 'Казань': (55.7823547, 49.1242266),
 'Красноярск': (56.0090968, 92.8725147),
 'Москва': (55.7504461, 37.6174943),
 'Новгород': (58.5209862, 31.2757862),
 'Новосибирск': (54.96781445, 82.95159894278376),
 'Ростов-на-Дону': (47.2213858, 39.7114196),
 'Самара': (53.198627, 50.113987),
 'Санкт-Петербург': (59.938732, 30.316229),
 'Уфа': (54.7261409, 55.947499)}

In [ ]:
from geopy import distance

In [ ]:
#расстояние считаю в метрах
def dist_to_cent(X):
  X['dist_to_cent'] = 0
  for i in range(X.shape[0]):
    X['dist_to_cent'][i] = distance.geodesic((X.lat[i],X.lon[i]), cities_centers[X.city[i]]).m
  return X

In [ ]:
test = dist_to_cent(test)
train = dist_to_cent(train)

### 3.2) Расстояние до ближайшей станции (здесь станция - жд или метро)

П.С. Не разобрался, как можно разделить метро и жд, потому что по тэгу building = train_station пустой ответ.

In [ ]:
# создаю словарь словарей для городов, в котором записан джсон полученый по апи запросу к ОСМ
merged_osm={'Москва':msc_osm,'Екатеринбург':ekb_osm,'Казань':kzn_osm,'Красноярск':krsn_osm,'Новгород':ngor_osm,'Новосибирск':nsib_osm,'Ростов-на-Дону':rnd_osm,
            'Самара':sam_osm,'Санкт-Петербург':spb_osm,'Уфа':ufa_osm}

In [ ]:
#расстояние считаю в метрах
def min_dist_to_station(X):
  X['min_dist_to_station'] = 0
  for i in range(X.shape[0]):
    current_city_osm = merged_osm[X.city[i]]
    dist = []
    for station in current_city_osm['station']['elements']:
      dist.append(distance.geodesic((X.lat[i],X.lon[i]),(station['lat'],station['lon'])).m)
    X['min_dist_to_station'][i] = min(dist)
  return X

In [ ]:
train = min_dist_to_station(train)
test = min_dist_to_station(test)

### 3.3-3.7) Определение количества станций, хостелов, достопримечательностей, точек фаст-фуда, ТЦ в буферной зоне вокруг точки продаж.

In [ ]:
from shapely.geometry import Point

Для этого буду строить вокруг точки буффер требуемого радиуса, предварительно нужно будет перевести радиус из метров в градусы, тк координаты точки задаю в градусах.

In [ ]:
import math
Rpo = 6356752 # meters
Req = 6378138 # meters
def radius(fi):
  nominator = pow(pow(Req,2)*math.cos(fi),2)+ pow(pow(Rpo,2)*math.sin(fi),2)
  denominator = pow(Req*math.cos(fi),2) + pow(Rpo*math.sin(fi),2)
  return round(math.sqrt(nominator/denominator))

Далее делаю допущение что длина дуги сектора равна радиусу буферной зоны (думаю, это можно сделать, тк радиусы большие). Дальше нахожу угол по школьной формуле.

In [ ]:
def angle(fi,buff_rad):
  r= radius(fi)
  half_alph = buff_rad/2 * 180/math.pi/r
  return round(2*half_alph,3)

In [ ]:
def fast_food_count(X,eps):
  X['fast_food_count'] = 0
  for i in range(X.shape[0]):
    current_city_osm = merged_osm[X.city[i]]
    salon = Point(X.lat[i],X.lon[i]).buffer(angle(X.lat[i],eps))
    for fastfood in current_city_osm['fast_food']['elements']:
      fastfood_point = Point(fastfood['lat'],fastfood['lon'])
      if salon.contains(fastfood_point):
        X['fast_food_count'][i] = X['fast_food_count'][i] + 1
  return X


In [ ]:
test = fast_food_count(test,200)
train = fast_food_count(train,200)

Количество станций в нужном радиусе

In [ ]:
def station_count(X,eps):
  X['station_count'] = 0
  for i in range(X.shape[0]):
    current_city_osm = merged_osm[X.city[i]]
    salon = Point(X.lat[i],X.lon[i]).buffer(angle(X.lat[i],eps))
    for fastfood in current_city_osm['station']['elements']:
      fastfood_point = Point(fastfood['lat'],fastfood['lon'])
      if salon.contains(fastfood_point):
        X['station_count'][i] = X['station_count'][i] + 1
  return X


In [ ]:
train = station_count(train,500) # радиус 500м
test = station_count(test,500) # радиус 500м

Количество хостелов

In [ ]:
def hostel_count(X,eps):
  X['hostel_count'] = 0
  for i in range(X.shape[0]):
    current_city_osm = merged_osm[X.city[i]]
    salon = Point(X.lat[i],X.lon[i]).buffer(angle(X.lat[i],eps))
    for fastfood in current_city_osm['hostel']['elements']:
      fastfood_point = Point(fastfood['lat'],fastfood['lon'])
      if salon.contains(fastfood_point):
        X['hostel_count'][i] = X['hostel_count'][i] + 1
  return X


In [ ]:
train = hostel_count(train,1000)
test = hostel_count(test,1000)

Количество ТЦ

In [ ]:
def mall_count(X,eps):
  X['mall_count'] = 0
  for i in range(X.shape[0]):
    current_city_osm = merged_osm[X.city[i]]
    salon = Point(X.lat[i],X.lon[i]).buffer(angle(X.lat[i],eps))
    for fastfood in current_city_osm['mall']['elements']:
      fastfood_point = Point(fastfood['lat'],fastfood['lon'])
      if salon.contains(fastfood_point):
        X['mall_count'][i] = X['mall_count'][i] + 1
  return X

In [ ]:
train = mall_count(train,5000)
test = mall_count(test,5000)

Количество достопримечательностей

In [ ]:
def att_count(X,eps):
  X['att_count'] = 0
  for i in range(X.shape[0]):
    current_city_osm = merged_osm[X.city[i]]
    salon = Point(X.lat[i],X.lon[i]).buffer(angle(X.lat[i],eps))
    for fastfood in current_city_osm['attraction']['elements']:
      fastfood_point = Point(fastfood['lat'],fastfood['lon'])
      if salon.contains(fastfood_point):
        X['att_count'][i] = X['att_count'][i] + 1
  return X

In [ ]:
train = att_count(train,5000)
test = att_count(test,5000)

## Визуализация

In [ ]:
import folium
map = folium.Map(location = cities_centers['Москва'])
visualization = train[train.city == "Москва"].reset_index(drop = True)
for i in range(visualization.shape[0]):
  folium.Marker([visualization.lat[i], visualization.lon[i]], popup="<i>Megafon</i>", tooltip="Buy phone!",
                icon=folium.Icon(color="green", icon="phone")).add_to(map)
moscow_osm = merged_osm['Москва']
for att in merged_osm['Москва']['attraction']['elements']:
  folium.Marker([att['lat'], att['lon']], popup="<i>Attraction</i>",
                icon=folium.Icon(color="blue", icon="info-sign")).add_to(map)
map.save("./data/map.html")
map


In [ ]:
visualization

,index,point_id,lon,lat,target,city,dist_to_cent,min_dist_to_station,fast_food_count,station_count,hostel_count,mall_count,att_count
0,0,ommNZCUV,37.590776,55.848630,-0.348157,Москва,11059,167,1,1,1,0,7
1,1,nMe2LHPb,37.784210,55.750271,1.294206,Москва,10468,176,5,1,0,0,8
2,3,0t2jNYdz,37.704570,55.782020,-1.169339,Москва,6498,59,11,2,0,0,10
3,4,U27W4QJ7,37.643983,55.730188,-0.088837,Москва,2802,221,5,1,7,2,36
4,7,CO76tdVs,37.444304,55.850511,0.213704,Москва,15559,283,13,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,420,AA8hN7bJ,37.628765,55.740664,0.213704,Москва,1298,87,4,3,10,2,51
156,421,uNw6t6xk,37.586825,55.794233,-0.002397,Москва,5241,167,5,3,3,4,93
157,422,Ap42ei8k,37.678790,55.772910,-0.910019,Москва,4589,113,5,1,5,0,11
158,423,rn9A8r62,37.752130,55.619640,-0.326547,Москва,16846,552,2,0,0,2,0


In [ ]:
from folium.plugins import HeatMap

In [ ]:
map_target = folium.Map(location = cities_centers['Москва'])
data = visualization.iloc[:,2:5]
min_tar = min(data.target)
max_tar = max(data.target)
heat_data =[]
for i in range(data.shape[0]):
  heat_data.append([data.lat[i], data.lon[i], data.target[i]])
heat_map = HeatMap(heat_data,radius = 15).add_to(map_target)
map_target.save('./data/map_target.html')
map_target

## Fit model

In [ ]:
train_copy = train.copy(deep =True)
test_copy = test.copy(deep = True)

In [ ]:
test.head(2)

,index,point_id,lon,lat,target,city,dist_to_cent,min_dist_to_station,fast_food_count,station_count,hostel_count,mall_count,att_count
0,0,F4lXR1cG,37.681242,55.74804,0.0091,Москва,4012,162,9,2,2,0,11
1,1,4LJu4GTf,60.580910,56.79586,0.0091,Екатеринбург,5096,2210,3,0,0,9,4


In [ ]:
to_drop =['index','city','point_id']

In [ ]:
train.drop(labels=to_drop,axis = 1, inplace=True)

In [ ]:
test.drop(labels=to_drop,axis =1, inplace= True)

In [ ]:

X_train, X_valid, y_train, y_valid = train_test_split(train.drop('target', axis=1), train[['target']])
model = LinearRegression().fit(X_train, y_train)

In [ ]:
mean_absolute_error(y_valid, model.predict(X_valid))

0.6256878473789654

## Make submission

In [ ]:
submit = pd.DataFrame(test_copy['point_id'])

In [ ]:
submit['target'] = model.predict(test.drop('target',axis = 1))

In [ ]:
submit.to_csv('./data/submition.csv', encoding = 'utf-8', index=False)